# Interactive Mesh Viewer
Select any generated mesh and explore it in a Plotly scene with adjustable decimation for faster rendering.

In [ ]:
from __future__ import annotations
from pathlib import Path
from typing import List
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import trimesh
from IPython.display import Markdown, display

REPO_ROOT = Path.cwd()
ARTIFACT_ROOT = REPO_ROOT / "artifacts"

def discover_meshes(root: Path) -> List[Path]:
    if not root.exists():
        return []
    patterns = ("*.glb", "*.obj")
    files: List[Path] = []
    for pattern in patterns:
        files.extend(root.rglob(pattern))
    unique: List[Path] = []
    seen = set()
    for path in sorted(files):
        resolved = path.resolve()
        if resolved in seen:
            continue
        seen.add(resolved)
        unique.append(resolved)
    return unique

def reduce_mesh_faces(mesh: trimesh.Trimesh, max_faces: int) -> trimesh.Trimesh:
    if max_faces <= 0 or mesh.faces.shape[0] <= max_faces:
        return mesh
    ratio = max_faces / mesh.faces.shape[0]
    try:
        simplified = mesh.simplify_quadric_decimation(max_faces)
        if simplified.faces.shape[0] > 0:
            return simplified
    except:
        pass
    step = max(1, mesh.faces.shape[0] // max_faces)
    face_ids = np.arange(0, mesh.faces.shape[0], step)[:max_faces]
    if face_ids.size < 3:
        return mesh
    reduced = mesh.submesh([face_ids], append=True, repair=False)
    if reduced is None or reduced.faces.size == 0:
        return mesh
    reduced.remove_unreferenced_vertices()
    return reduced

def format_mesh_label(path: Path) -> str:
    for base in (ARTIFACT_ROOT, REPO_ROOT):
        try:
            return path.relative_to(base).as_posix()
        except ValueError:
            continue
    return path.name

In [9]:
AVAILABLE_MESHES = discover_meshes(ARTIFACT_ROOT)
if not AVAILABLE_MESHES:
    display(Markdown("No meshes found under `artifacts/`. Run `scripts/mesh_from_mask.py` first."))
else:
    mesh_options = [
        (format_mesh_label(path), str(path)) for path in AVAILABLE_MESHES
    ]
    mesh_dropdown = widgets.Dropdown(
        options=mesh_options,
        description="Mesh:",
        layout=widgets.Layout(width="80%"),
    )
    faces_slider = widgets.IntSlider(
        value=50000,
        min=5000,
        max=200000,
        step=5000,
        description="Max faces",
        continuous_update=False,
    )

    def render_mesh(mesh_path: str, max_faces: int) -> None:
        if not mesh_path:
            return
        path = Path(mesh_path)
        loaded = trimesh.load(path, force="mesh")
        if isinstance(loaded, trimesh.Scene):
            concatenated = [g for g in loaded.geometry.values()]
            mesh = trimesh.util.concatenate(concatenated) if concatenated else None
        else:
            mesh = loaded
        if mesh is None or not hasattr(mesh, 'vertices'):
            display(Markdown(f"Unable to load geometry from **{path.name}**."))
            return
        
        original_faces = mesh.faces.shape[0]
        original_verts = mesh.vertices.shape[0]
        
        mesh = mesh.copy()
        if mesh.vertices.size == 0 or mesh.faces.size == 0:
            display(Markdown("The selected mesh contains no triangles to visualize."))
            return
        
        mesh.remove_unreferenced_vertices()
        mesh.apply_translation(-mesh.centroid)
        
        simplified = reduce_mesh_faces(mesh, max_faces)
        vertices = simplified.vertices
        faces = simplified.faces
        
        if vertices.size == 0 or faces.size == 0:
            display(Markdown("Mesh could not be simplified for visualization."))
            return
        
        bbox = vertices.ptp(axis=0)
        stats = (
            f"**{path.name}**  ",
            f"Original: {original_verts:,} vertices / {original_faces:,} faces  ",
            f"Simplified: {vertices.shape[0]:,} vertices / {faces.shape[0]:,} faces  ",
            f"Bounding box: {bbox[0]:.2f} × {bbox[1]:.2f} × {bbox[2]:.2f}",
        )
        display(Markdown("\n".join(stats)))
        
        colors = None
        if hasattr(simplified, 'visual') and hasattr(simplified.visual, 'vertex_colors'):
            colors = simplified.visual.vertex_colors[:, :3] / 255.0
        
        mesh_trace = go.Mesh3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            i=faces[:, 0],
            j=faces[:, 1],
            k=faces[:, 2],
            lighting=dict(
                ambient=0.5,
                diffuse=0.8,
                specular=0.3,
                roughness=0.5,
                fresnel=0.2
            ),
            lightposition=dict(x=100, y=200, z=50),
            flatshading=False,
            opacity=1.0,
        )
        
        if colors is not None and colors.shape[0] == vertices.shape[0]:
            mesh_trace.vertexcolor = colors
        else:
            mesh_trace.colorscale = "Viridis"
            mesh_trace.intensity = vertices[:, 2]
            mesh_trace.showscale = False
        
        fig = go.Figure(data=[mesh_trace])
        fig.update_layout(
            title=f"Interactive view • {path.name}",
            scene=dict(
                aspectmode="data",
                camera=dict(
                    eye=dict(x=1.5, y=1.5, z=1.5),
                    center=dict(x=0, y=0, z=0)
                ),
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            ),
            margin=dict(l=0, r=0, t=35, b=0),
            width=900,
            height=700,
        )
        fig.show()

    controls = widgets.VBox([mesh_dropdown, faces_slider])
    output = widgets.interactive_output(
        render_mesh, {"mesh_path": mesh_dropdown, "max_faces": faces_slider}
    )
    display(controls, output)

Output()

In [10]:
try:
    import pyvista as pv
    from matplotlib import pyplot as plt
    
    AVAILABLE_MESHES_PV = discover_meshes(ARTIFACT_ROOT)
    if not AVAILABLE_MESHES_PV:
        display(Markdown("No meshes found under `artifacts/`. Run `scripts/mesh_from_mask.py` first."))
    else:
        mesh_options_pv = [
            (format_mesh_label(path), str(path)) for path in AVAILABLE_MESHES_PV
        ]
        mesh_dropdown_pv = widgets.Dropdown(
            options=mesh_options_pv,
            description="Mesh:",
            layout=widgets.Layout(width="80%"),
        )
        num_views_slider = widgets.IntSlider(
            value=8,
            min=4,
            max=16,
            step=4,
            description="Views",
            continuous_update=False,
        )
        
        def render_turntable(mesh_path: str, num_views: int) -> None:
            if not mesh_path:
                return
            
            path = Path(mesh_path)
            display(Markdown(f"**Generating {num_views} views for {path.name}...**"))
            
            # Load mesh with PyVista
            loaded = pv.read(mesh_path)
            
            # Handle MultiBlock (GLB files with multiple meshes)
            if isinstance(loaded, pv.MultiBlock):
                meshes = [block for block in loaded if block is not None]
                if not meshes:
                    display(Markdown("No valid meshes found in file."))
                    return
                # Combine all meshes into one
                pv_mesh = meshes[0]
                for mesh in meshes[1:]:
                    pv_mesh = pv_mesh.merge(mesh)
            else:
                pv_mesh = loaded
            
            # Center the mesh
            center = np.array(pv_mesh.center)
            pv_mesh.translate(-center, inplace=True)
            
            # Calculate camera distance based on bounds
            bounds = pv_mesh.bounds
            max_dim = max(bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])
            camera_distance = max_dim * 2.5
            
            # Create figure for grid of views
            cols = 4
            rows = (num_views + cols - 1) // cols
            fig, axes = plt.subplots(rows, cols, figsize=(16, 4 * rows))
            axes = axes.flatten() if num_views > 1 else [axes]
            
            # Generate views from different angles
            angles = np.linspace(0, 360, num_views, endpoint=False)
            
            for idx, angle in enumerate(angles):
                plotter = pv.Plotter(off_screen=True, window_size=[512, 512])
                plotter.add_mesh(
                    pv_mesh,
                    smooth_shading=True,
                    color='white',
                    show_edges=False,
                    lighting=True
                )
                
                # Position camera
                angle_rad = np.deg2rad(angle)
                camera_pos = [
                    camera_distance * np.cos(angle_rad),
                    camera_distance * np.sin(angle_rad),
                    camera_distance * 0.3
                ]
                plotter.camera_position = [
                    camera_pos,
                    [0, 0, 0],
                    [0, 0, 1]
                ]
                
                # Add lighting
                plotter.add_light(pv.Light(position=(10, 10, 10), light_type='scene light'))
                
                # Render to image
                img = plotter.screenshot(return_img=True)
                plotter.close()
                
                # Display in subplot
                axes[idx].imshow(img)
                axes[idx].set_title(f'Azimuth: {int(angle)}°', fontsize=10)
                axes[idx].axis('off')
            
            # Hide unused subplots
            for idx in range(num_views, len(axes)):
                axes[idx].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Display mesh statistics
            stats = (
                f"**Statistics**  ",
                f"Vertices: {pv_mesh.n_points:,}  ",
                f"Faces: {pv_mesh.n_cells:,}  ",
                f"Bounds: [{bounds[0]:.2f}, {bounds[1]:.2f}] × [{bounds[2]:.2f}, {bounds[3]:.2f}] × [{bounds[4]:.2f}, {bounds[5]:.2f}]",
            )
            display(Markdown("\n".join(stats)))
        
        controls_pv = widgets.VBox([mesh_dropdown_pv, num_views_slider])
        output_pv = widgets.interactive_output(
            render_turntable, {"mesh_path": mesh_dropdown_pv, "num_views": num_views_slider}
        )
        display(controls_pv, output_pv)

except ImportError:
    display(Markdown("""
    **PyVista not installed.** Install it with:
    ```bash
    pip install pyvista matplotlib
    ```
    """))

Output()

# Multi-View Consistency Checker (PyVista)
Generate turntable renders from multiple viewpoints to inspect mesh consistency. This creates a grid showing the mesh from 8 evenly-spaced angles around the object.